[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/OpenAccess.ipynb)

# Open Acces Analysis
Based in http://oadoi.org full DataBase

## Load UDEA_DOIS

In [0]:
!pip install wosplus openpyxl currencyconverter > /dev/null

In [118]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx   = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj

Writing drive.cfg


In [0]:
import wosplus as wp
import numpy as np
import pandas as pd
import os

In [0]:
files=wp.wosplus('drive.cfg')

In [0]:
dois=files.read_drive_excel('DOIS.xlsx')

## Normalize DOIS

In [0]:
dois['DI']=dois['DI'].str.lower()

## Load oadoi.org data

In [13]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
WORK_DIR="drive/My Drive/Colab Notebooks/Data"
os.chdir(WORK_DIR)

In [0]:
oa=pd.read_csv('oadoi.csv')

In [0]:
oa['DOI']=oa['DOI'].str.lower()

In [0]:
yellow=pd.read_csv('oayellowdois.csv')
yellow['DOI']=yellow['DOI'].str.lower()

In [0]:
green=pd.read_csv('oagreendois.csv')
green['DOI']=green['DOI'].str.lower()

In [0]:
greenarticle=pd.read_csv('oagreenarticledois.csv')
greenarticle['DOI']=greenarticle['DOI'].str.lower()

## Merge

In [0]:
oafdois=dois.merge(oa,left_on='DI',right_on='DOI',how='left')

In [0]:
oafdois=oafdois.fillna('')

In [95]:
oafdois[oafdois.DOI!=''].shape

(3491, 2)

In [0]:
oafdois=oafdois.rename({'DOI':'Open_Access'}, axis = 'columns')

In [0]:
oayellowdois=oafdois.merge(yellow,left_on='DI',right_on='DOI',how='left')
oayellowdois=oayellowdois.fillna('')
oayellowdois[oayellowdois.DOI!=''].shape
oayellowdois=oayellowdois.rename({'DOI':'OA_Yellow'}, axis = 'columns')

In [98]:
oayellowdois[oayellowdois['OA_Yellow']!=''].shape

(1505, 3)

In [0]:
oaygdois=oayellowdois.merge(green,left_on='DI',right_on='DOI',how='left')
oaygdois=oaygdois.fillna('')
oaygdois=oaygdois.rename({'DOI':'OA_Green'}, axis = 'columns')

In [104]:
oaygdois[oaygdois['OA_Green']!=''].shape

(1726, 4)

In [101]:
oaygdois[np.logical_or(oaygdois.OA_Yellow!='' ,oaygdois.OA_Green!='')].shape

(3231, 4)

In [0]:
oadois=oaygdois.merge(greenarticle,left_on='DI',right_on='DOI',how='left')
oadois=oadois.fillna('')
oadois=oadois.rename({'DOI':'OA_Green_Article'}, axis = 'columns')

In [109]:
oadois[oadois['OA_Green_Article']!=''].shape

(260, 5)

In [112]:
oadois[np.logical_or( np.logical_or(oadois.OA_Yellow!='' ,oadois.OA_Green!=''),oadois.OA_Green_Article!='')].shape

(3491, 5)

In [0]:
oadois.to_excel('oaudea.xlsx',index=False)

In [116]:
260+1726+1505

3491

In [117]:
260+1726

1986

In [132]:
oadois[:1]

,DI,Open_Access,OA_Yellow,OA_Green,OA_Green_Article
0,10.7705/biomedica.v37i3.4058,10.7705/biomedica.v37i3.4058,10.7705/biomedica.v37i3.4058,,


In [133]:
oadois.shape

(7993, 5)

## Check with SCP

In [0]:
udea=files.read_drive_excel('UDEA_WOS_SCI_SCP.xlsx')
udea=wp.fill_NaN(udea)

In [0]:
udea['DI']=udea['DI'].str.lower()

In [0]:
oascp=udea[udea['SCP_Access Type']=='Open Access'].reset_index(drop=True)

In [129]:
oascp[oascp['DI']==''].SO

10699    REVISTA BRASILEIRA DE FARMACOGNOSIA-BRAZILIAN ...
10714             BRAZILIAN JOURNAL OF INFECTIOUS DISEASES
10764                     Revista de la educación superior
10781                                     Gaceta Sanitaria
11124                                    ATENCION PRIMARIA
11301                 Revista Colombiana de Anestesiología
11302                 Revista Colombiana de Anestesiología
11303                 Revista Colombiana de Anestesiología
11737                 Revista Colombiana de Anestesiología
11741                 Revista Colombiana de Anestesiología
11843                      Revista argentina de radiología
11855           Corpoica Ciencia y Tecnología Agropecuaria
11917                    Revista Colombiana de Cardiología
12456           Corpoica Ciencia y Tecnología Agropecuaria
12457           Corpoica Ciencia y Tecnología Agropecuaria
12479                     Revista de la educación superior
12519                                             Infect

## Quality check:  
Expected 0

In [0]:
kk=oascp[oascp['OA_SCP']!=''][['OA_SCP']].merge(oadois,left_on='OA_SCP',right_on='Open_Access',how='left')

In [151]:
kk[kk['Open_Access']==''].shape

(0, 6)

## APC

In [210]:
udea[ udea['DI'].str.lower().str.contains('10.1109/cerma.2010.131') ][['DI','SN','SCI_SN','SCP_ISSN','ISSN']]

,DI,SN,SCI_SN,SCP_ISSN,ISSN
8252,10.1109/cerma.2010.131,,,,


In [199]:
udea['SN'][:1]

0    0002-9378\n
Name: SN, dtype: object

In [200]:
udea[ udea['SCI_SN']!='' ].SCI_SN[:1]

1186    2175-7968\n
Name: SCI_SN, dtype: object

In [0]:
udea['ISSN']=( udea['SN']+udea['SCI_SN']+udea['SCP_ISSN'].str.replace('(^[a-zA-Z0-9]{4})([a-zA-Z0-9]{4}$)',r'\1-\2\n')).str.split('\n').str[0]

In [202]:
oadois[:1]

,DI,Open_Access,OA_Yellow,OA_Green,OA_Green_Article
0,10.7705/biomedica.v37i3.4058,10.7705/biomedica.v37i3.4058,10.7705/biomedica.v37i3.4058,,


In [0]:
udeadois=oadois.merge( udea[['DI','ISSN']],on='DI',how='left')  

In [0]:
udeadois=udeadois.fillna('')

## Search APC  in DOAJ

In [0]:
import random

In [0]:
def get_APC(issn):
    from bs4 import BeautifulSoup
    import requests
    import random
    import time

    url='https://doaj.org/toc/{}'.format(issn)
    r=requests.get(url)
    soup=BeautifulSoup(r.text)
    try:
        APC=soup.find_all('div',{"class":"box slim"})[0].contents[3].contents[2].strip()
    except:
        APC=''
        
    time.sleep(random.randint(1,3))    
    return APC

In [221]:
sn='2045-2322'#'0960-9822'#'2211-3797'
get_APC(sn)

'1165GBP'

In [0]:
apcs=[]
for sn in udeadois[ udeadois['ISSN']!='' ].ISSN.unique():#[:4]:
    apcs.append({'ISSN':sn,'APC':get_APC(sn)})

In [0]:
apcf=pd.DataFrame( apcs )
apcf=apcf.fillna('')

In [0]:
apc=apcf[apcf.APC!=''].reset_index(drop=True)

In [241]:
apc[:2]

,APC,ISSN
0,550GBP,2049-1891
1,4000USD,2373-8057


In [0]:
!pip install currencyconverter > /dev/null

In [0]:
import re
from currency_converter import CurrencyConverter
c = CurrencyConverter()

def usd(APC):
    import sys
    apcusd=0
    i=re.search(r"([0-9\.\,]+)([a-zA-Z]+)",APC)
    items=i.groups()
    if len(items)==2:
        try:
            apcusd=c.convert(float(items[0]),items[1],'USD')

        #capture error message: https://stackoverflow.com/a/4690655        
        except ValueError as e: 
            #Missing currencies
            COP_to_USD=1/3000.
            ARS_to_USD=1/36.2
            if str(e).split()[0]=='COP':
                apcusd=float(items[0])*COP_to_USD
            elif str(e).split()[0]=='ARS':
                apcusd=float(items[0])*ARS_to_USD
            else:
                sys.exit(e)
    else:
        apcusd=0
            
    return int( apcusd  )

In [300]:
usd( '5000ARS' )

138

In [0]:
apc['APC_USD']=apc.APC.apply(usd)

In [0]:
UDEADOIS=udeadois.merge(apc,on='ISSN',how='left')

In [0]:
UDEADOIS=wp.fill_NaN( UDEADOIS )

In [0]:
UDEADOIS.to_excel('oaudea.xlsx',index=False)

### Quality check
Only Open Access have APCs

In [307]:
tmp=UDEADOIS[UDEADOIS['Open_Access']=='']
tmp[ tmp['APC']!=''].shape # expected 0

(24, 8)

In [0]:
OAUDEA=UDEADOIS[ np.logical_or (UDEADOIS['OA_Green']!='',UDEADOIS['OA_Green_Article']!='') ]

In [321]:
OAUDEA[OAUDEA['APC']!='']

(627, 8)

In [323]:
OAUDEA.APC_USD.sum()

920276.0

## Calculates APC

In [0]:
UDEADOIS['APC_USD'].sum()

# Consultar  DOAJ API con DOI
Ver: https://doaj.org/api/v1/docs#!/Search/get_api_v1_search_articles_search_query

## Carga navegador JSON

In [0]:
import pandas as pd

## Consulta

In [0]:
DOI='10.1016/j.physletb.2018.01.009'
df=pd.read_json('https://doaj.org/api/v1/search/articles/{}'.format(DOI))

## Análisis de resultado:
Nota: La celdas con estructuras anidadas (que comienzan con `[` ó `{`) sa analizan secuencialmente

In [0]:
df

,last,page,pageSize,query,results,timestamp,total
0,https://doaj.org/api/v1/search/articles/10.101...,1,10,10.1016/j.physletb.2018.01.009,"{u'last_updated': u'2018-03-05T12:09:16Z', u'i...",2018-1010T04:23:08Z,1


Desplegar colunna __results__:

In [0]:
p=pd.DataFrame(  df['results'].values[0] )
p

,bibjson,created_date,id,last_updated
abstract,The observation of neutrino oscillations estab...,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
author,"[{u'affiliation': u'Physics Department, Univer...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
end_page,100,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
identifier,"[{u'type': u'pissn', u'id': u'0370-2693'}, {u'...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
journal,"{u'publisher': u'Elsevier', u'language': [u'EN...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
link,[{u'url': u'http://www.sciencedirect.com/scien...,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
month,3,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
start_page,94,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
subject,"[{u'code': u'QC1-999', u'term': u'Physics', u'...",2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z
title,Expanding the reach of heavy neutrino searches...,2018-03-05T12:09:16Z,03ba3f288d8341d380a286a6d0f64de4,2018-03-05T12:09:16Z


Desplegar cada una de las celdas anidadas de la columna __bibjson__

In [0]:
pd.DataFrame(  p.loc['author','bibjson'] )

,affiliation,name
0,"Physics Department, Universidad de los Andes, ...",Andrés Flórez
1,"Department of Physics and Astronomy, Vanderbil...",Kaiwen Gui
2,"Department of Physics and Astronomy, Vanderbil...",Alfredo Gurrola
3,"Physics Department, Universidad de los Andes, ...",Carlos Patiño
4,"Department of Physics, Universidad de Antioqui...",Diego Restrepo


In [0]:
pd.DataFrame(  p.loc['identifier','bibjson'] )

,id,type
0,0370-2693,pissn
1,1873-2445,eissn
2,10.1016/j.physletb.2018.01.009,doi


In [0]:
 pd.DataFrame( [p.loc['journal','bibjson'] ] )


,country,issns,language,license,number,publisher,title,volume
0,NL,"[0370-2693, 1873-2445]",[EN],[{u'url': u'http://www.elsevier.com/journals/p...,C,Elsevier,Physics Letters B,778


In [0]:
pd.DataFrame( p.loc['journal','bibjson']['license'] )

,open_access,title,type,url
0,True,CC BY,CC BY,http://www.elsevier.com/journals/physics-lette...


In [0]:
pd.DataFrame(  p.loc['link','bibjson']  )

,type,url
0,fulltext,http://www.sciencedirect.com/science/article/p...
